In [2]:
import __init__

server location local


running from: Local


In [3]:
import json
from joblib import dump, load
import pandas as pd
import main
from stocknews_api import StockNewsAPI
from scrapy import Scrapy

In [2]:
token = load("./stocknewsapi/stocknewsapi_key.pkl")

In [3]:
start_date = "2019-01-01"
end_date = "2019-12-31"
tickers = "AAPL"
file_name = './data/stock_news_api_' + start_date + '_' + end_date + '.pkl'

In [4]:
stocknewsapi = StockNewsAPI(token, start_date, end_date)

In [5]:
stocknewsapi_news = stocknewsapi.get_news(tickers, items=50, pages=1)

[ 100% ] https://stocknewsapi.com/api/v1?tickers=AAPL&type=article&token=timo8bdrdfcpfwweljogcb66iodw7egybu0y6qsw&fallback=true&country=USA&items=50&sortby=rank&pages=1&date=12232019-12272019

In [6]:
dump(stocknewsapi_news, file_name, compress=3)

['./data/stock_news_api_2019-01-01_2019-12-31.pkl']

In [12]:
start_date = "2020-01-01"
end_date = "2020-12-31"
tickers = "AAPL"
file_name = './data/stock_news_api_' + start_date + '_' + end_date + '.pkl'

In [13]:
stocknewsapi = StockNewsAPI(token, start_date, end_date)

In [14]:
stocknewsapi_news = stocknewsapi.get_news(tickers, items=50, pages=1)

[ 100% ] https://stocknewsapi.com/api/v1?tickers=AAPL&type=article&token=timo8bdrdfcpfwweljogcb66iodw7egybu0y6qsw&fallback=true&country=USA&items=50&sortby=rank&pages=1&date=12222020-12262020

In [15]:
dump(stocknewsapi_news, file_name, compress=3)

['./data/stock_news_api_2020-01-01_2020-12-31.pkl']

In [3]:
start_date = "2021-01-01"
end_date = "2021-04-30"
tickers = "AAPL"
file_name = './data/stock_news_api_' + start_date + '_' + end_date + '.pkl'

In [4]:
stocknewsapi = StockNewsAPI(token, start_date, end_date)

In [5]:
stocknewsapi_news = stocknewsapi.get_news(tickers, items=50, pages=1)

[ 100% ] https://stocknewsapi.com/api/v1?tickers=AAPL&type=article&token=timo8bdrdfcpfwweljogcb66iodw7egybu0y6qsw&fallback=true&country=USA&items=50&sortby=rank&pages=1&date=04232021-04272021

In [8]:
dump(stocknewsapi_news, file_name, compress=3)

['./data/stock_news_api_2021-01-01_2021-04-30.pkl']

In [3]:
apple_news = list()
apple_news_2019 = load('./data/stock_news_api_2019-01-01_2019-12-31.pkl')
apple_news_2020 = load('./data/stock_news_api_2020-01-01_2020-12-31.pkl')
apple_news_2021 = load('./data/stock_news_api_2021-01-01_2021-04-30.pkl')

In [4]:
apple_news.extend(apple_news_2019)
apple_news.extend(apple_news_2020)
apple_news.extend(apple_news_2021)

In [8]:
apple_news_df = pd.DataFrame(apple_news)

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers
0,https://www.cnbc.com/2021/05/25/home-prices-in...,https://cdn.snapi.dev/images/v1/b/m/104669168-...,Home prices in March saw highest growth in ove...,"Home prices in March were up 13%, one of the l...",CNBC,"Tue, 25 May 2021 09:01:19 -0400",[],Neutral,Article,NaN
1,https://www.reuters.com/article/us-usa-economy...,https://cdn.snapi.dev/images/v1/a/s/m02d202105...,U.S. home prices rose more than expected in Ma...,U.S. single-family home prices in 20 key urban...,Reuters,"Tue, 25 May 2021 09:09:00 -0400",[],Neutral,Article,NaN
2,https://www.reuters.com/article/us-usa-economy...,https://cdn.snapi.dev/images/v1/8/c/m02d202105...,U.S. new home sales drop in April; March sales...,Sales of new U.S. single-family homes dropped ...,Reuters,"Tue, 25 May 2021 10:33:00 -0400",[],Negative,Article,NaN


In [13]:
apple_news_df.to_csv('./data/apple_news.csv')

In [3]:
apple_news_df = pd.read_csv('./data/apple_news.csv')

In [4]:
articles, missing_articles = Scrapy.scrap(apple_news_df.news_url)

In [10]:
dump(articles, './data/articles.pkl', compress=5)

['./data/articles.pkl']

In [6]:
apple_news_df['articles'] = articles

In [7]:
apple_news_df.to_csv('./data/apple_news_scraped.csv')

In [4]:
apple_news_df = pd.read_csv('./data/apple_news_scraped.csv')

In [5]:
apple_news_df.head(3)

,Unnamed: 0,Unnamed: 0.1,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers,articles
0,0,0,https://www.cnbc.com/2021/05/25/home-prices-in...,https://cdn.snapi.dev/images/v1/b/m/104669168-...,Home prices in March saw highest growth in ove...,"Home prices in March were up 13%, one of the l...",CNBC,"Tue, 25 May 2021 09:01:19 -0400",[],Neutral,Article,NaN,Home prices in March were 13.2% higher in Mar...
1,1,1,https://www.reuters.com/article/us-usa-economy...,https://cdn.snapi.dev/images/v1/a/s/m02d202105...,U.S. home prices rose more than expected in Ma...,U.S. single-family home prices in 20 key urban...,Reuters,"Tue, 25 May 2021 09:09:00 -0400",[],Neutral,Article,NaN,U.S. single-family home prices in 20 key urban...
2,2,2,https://www.reuters.com/article/us-usa-economy...,https://cdn.snapi.dev/images/v1/8/c/m02d202105...,U.S. new home sales drop in April; March sales...,Sales of new U.S. single-family homes dropped ...,Reuters,"Tue, 25 May 2021 10:33:00 -0400",[],Negative,Article,NaN,A carpenter works on building new townhomes th...
